# 교차검증

In [2]:
# 테스트 세트를 사용하지 않고 모델이 과적합인지 과소적합인지 즉 과적합을 판단방법
# 보고서용....... 데이터가 작거나... 또는 머신러닝을 수행을때 과적합이 발생하면...
# 먼저 교차검증으로 평균치를 보여주고 평균 이정도 성능을 보인다.... 명시한다음에..
# 파라메터 튜닝을 통해.. 좀더 수치를 끌어올려본다..
# 그래도 너무  성능이 안좋다....... 데이터를 의심해보자... 잘못수집되었거나. 
# 전처리도 ...... 
# 컬럼이 많다면.... 결정트리의 피처중요도를 이용해서 상위 몇개의 피처만 가지고 다시 수행.

In [3]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/bigdataleeky/python/main/wine.csv")
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
x = df.iloc[:,:-1]
x.head()

,alcohol,sugar,pH
0,9.4,1.9,3.51
1,9.8,2.6,3.20
2,9.8,2.3,3.26
3,9.8,1.9,3.16
4,9.4,1.9,3.51


In [5]:
y = df.iloc[:,-1]
y = y.astype(int)
y.value_counts()

1    4898
0    1599
Name: class, dtype: int64

In [6]:
# 최종테스트 데이터
from sklearn.model_selection import train_test_split
x_train,x_target,y_train,y_target = train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, y_train.shape

((5197, 3), (5197,))

In [7]:
# 훈련용데이터를 - 훈련용 + 검증용  8:2
x_train2,x_target2,y_train2,y_target2 =  train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [8]:
x_train2.shape, y_train2.shape

((4157, 3), (4157,))

In [9]:
from sklearn.tree import DecisionTreeClassifier
dc =  DecisionTreeClassifier(random_state=42)
dc.fit(x_train2, y_train2)
dc.score(x_train2, y_train2), dc.score(x_target2, y_target2)

(0.9971133028626413, 0.864423076923077)

In [10]:
# GridSearchCV --- > 하이퍼 파라메터 튜닝....  model_selection
from sklearn.model_selection import cross_validate
score =  cross_validate(dc,x_train, y_train)


In [11]:
score['test_score']

array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])

In [12]:
score['test_score'].mean()

0.855300214703487

In [13]:
import numpy as np
np.mean(score['test_score'])

0.855300214703487

# 교차검증의 장점
## 모든데이터 셋을 훈련과 평가에 활용
## 정확도를 향상
## 데이터가 부족할때 이로인한 과적합
## 데이터 편중을 막을수 있다
## 좀더 일반화된 모델을

# 단점
## 시간이 과다하게 소요된다.

In [14]:
# Kfold 방식되 거의 유사 
# 분류기를 통한 교차검증
# 분류모델이면 StratifiedKFold 사용하고 그 외에는 
# 예측..군집일경우에는 KFold
from sklearn.model_selection import StratifiedKFold
score =  cross_validate(dc,x_train, y_train, cv=StratifiedKFold() )
score['test_score'].mean()

0.855300214703487

In [15]:
dc.feature_names_in_, dc.feature_importances_

(array(['alcohol', 'sugar', 'pH'], dtype=object),
 array([0.23614177, 0.50084785, 0.26301038]))

In [16]:
dc.feature_importances_

array([0.23614177, 0.50084785, 0.26301038])

In [17]:
np.sort(dc.feature_importances_)[::-1]

array([0.50084785, 0.26301038, 0.23614177])

In [18]:
# 결정트리의 사용 이유 : 정확한 분류..
# 부가적인 기능 : 피어의 중요도를 알수 있다..
# 만약에 피처가 너무 많다면.... feature_importances_의 값을 내람차순으로 정렬한후 상위 몇개의 feature만 추출해서
# 원하는 머신러닝에 적용

In [19]:
# 교차검증으로 과적합을 피할수 있다면..... 성능을 높이기 위해서 하이퍼 파라메터 튜닝을통해 성능 확인
dc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [20]:
p ={
    'min_impurity_decrease':[0.0001,0.001,0.01,0.1,1,10,100]
}

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
gs = GridSearchCV(dc, param_grid=p)

In [23]:
gs.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_impurity_decrease': [0.0001, 0.001, 0.01, 0.1, 1,
                                                   10, 100]})

In [24]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [25]:
model = gs.best_estimator_

In [26]:
model.score(x_target,y_target)

0.8653846153846154

In [27]:
model.score(x_train, y_train), model.score(x_target, y_target)

(0.9615162593804117, 0.8653846153846154)

In [28]:
params = {
    'min_impurity_decrease':[0.0001,0.001,0.01,0.1,1,10,100],
    'max_depth' : [3,5]
}

In [29]:
gs = GridSearchCV(dc, param_grid=params)
gs.fit(x_train,y_train)
gs.best_params_

{'max_depth': 5, 'min_impurity_decrease': 0.001}

In [30]:
model = gs.best_estimator_
model.score(x_train, y_train), model.score(x_target, y_target)

(0.8664614200500289, 0.8615384615384616)

In [31]:
# 앙상블 학습알고리즘 : 정형데이터를 다루는데 성능이 가장 뛰어난(머신러닝..), 성능을 높이기 위해 여러개의 
# 모델을 사용
# 랜덤포레스트 알고리즘, 엑스트라 트리알고리즘ㅡ 그레이디언트 부스팅 알고리즘

# 랜덤포레스트 알고리즘  : 결정트리를 랜덤하게 만들어서 숲을 구성
# 랜덤하게 데이터를 추출하는 방법 : 부트스트랩 샘플

In [32]:
# 트리알고리즘은 정규화 안한다...... (StandardScaling) ...
df = pd.read_csv("https://raw.githubusercontent.com/bigdataleeky/python/main/wine.csv")
df['class'] = df['class'].astype(int)
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0
1,9.8,2.6,3.20,0
2,9.8,2.3,3.26,0
3,9.8,1.9,3.16,0
4,9.4,1.9,3.51,0


In [33]:
x =df.iloc[:,:-1]
y =df.iloc[:,-1]
x.shape, y.shape

((6497, 3), (6497,))

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
x_train,x_target,y_train,y_target =  train_test_split(x,y,random_state=42)
x_train.shape,y_train.shape

((4872, 3), (4872,))

In [36]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(x_train, y_train)
rfc.score(x_train, y_train), rfc.score(x_target, y_target) 

(0.9973316912972086, 0.8812307692307693)

In [37]:
score =  cross_validate(rfc,x_train,y_train,cv=StratifiedKFold() )
score

{'fit_time': array([0.22919273, 0.2202909 , 0.22327232, 0.22149801, 0.2274332 ]),
 'score_time': array([0.01998234, 0.02000475, 0.02000332, 0.01994729, 0.02194381]),
 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688])}

In [38]:
score['test_score'].mean()

0.8914208392565683

In [39]:
# 리포트
# 피처의 중요도를 알수....
# 랜덤포레스트를 사용하는데. 크로스벨리데이터를 이용해서 전체학습데이터를 사용했다.
# 피처의 중요도를 알아보기위해
rfc.fit(x_train,y_train)
rfc.feature_importances_

array([0.23155241, 0.49706658, 0.27138101])

In [40]:
# Tree 그 자체로 뭔가 구분도 할수도 있지만... Feature 엔지니어링에 활용

In [41]:
rfc2 = RandomForestClassifier(oob_score=True,random_state=42)
rfc2.fit(x_train,y_train)
rfc2.feature_importances_

array([0.23155241, 0.49706658, 0.27138101])


## 엑스트라 트리
## 랜덤포레스트 보다 무작위 성이 크다....
## 노드를 랜덤하게 분할하기때문에 계산속도가 빠르가

In [42]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier()
score = cross_validate(et,x_train, y_train)

In [43]:
score

{'fit_time': array([0.21147227, 0.19659948, 0.20106602, 0.19647431, 0.20001984]),
 'score_time': array([0.02493358, 0.02448702, 0.02593017, 0.02496171, 0.02792501]),
 'test_score': array([0.88717949, 0.89230769, 0.90041068, 0.88398357, 0.8788501 ])}

In [44]:
score['test_score'].mean()

0.8885463065339863

## 그레디언트 부스팅
## 깊이가 얖은 트리를 만들고 다음 트리는 이전트리의 결점을 보안하는 방식의 앙상블
## 깊이가3 결정트리를 100개 --> 과적합에 강하고 높은 일반화 성능 기대
## 경사하강법을  트리앙상블 적용.. 손실함수를 사용 분류: 로지스틱 함수
##                                                회귀 : 평균제곱오차 MSE

In [55]:
from sklearn.ensemble import GradientBoostingClassifier
gd =  GradientBoostingClassifier(random_state=42,learning_rate= 0.3)
score = cross_validate(gd,x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.876641288895909, 0.9146652520989097)

In [59]:
from sklearn.model_selection import GridSearchCV  # 튜닝
param = {
    'learning_rate':np.arange(0.1,1.5,0.1)
}
gs = GridSearchCV(gd,param_grid=param)
gs.fit(x_train,y_train)

GridSearchCV(estimator=GradientBoostingClassifier(learning_rate=0.3,
                                                  random_state=42),
             param_grid={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4])})

In [66]:
gs.best_params_

{'learning_rate': 0.4}

In [67]:
model = gs.best_estimator_
score = cross_validate(model,x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.8770504922866319, 0.9233888719704268)

## 히스토그램 기반 그레디언트 부스팅
## 정형데이터를 다루는 머신러닝 알고리즘중에 가장 인기가 많은
## X의 특성을 256개의 구간으로 나눔.... 트리를 구성할때 최적의 노드가 되도록 분할..  효율과 속도 up
## 256개중에서 한개를 y로 나머지는 x로 사용


In [64]:
#from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [65]:
hgbc =  HistGradientBoostingClassifier(random_state=42)
score =  cross_validate(hgbc, x_train, y_train,return_train_score=True)
score['train_score'].mean(),  score['test_score'].mean()

(0.9380129799494501, 0.8805410414363187)

In [71]:
hgbc.get_params()

{'categorical_features': None,
 'early_stopping': 'auto',
 'l2_regularization': 0.0,
 'learning_rate': 0.1,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 100,
 'max_leaf_nodes': 31,
 'min_samples_leaf': 20,
 'monotonic_cst': None,
 'n_iter_no_change': 10,
 'random_state': 42,
 'scoring': 'loss',
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [72]:
param = {
    'learning_rate':np.arange(0.1,1.5,0.1)
}
gs = GridSearchCV(hgbc,param_grid=param)
gs.fit(x_train,y_train)

GridSearchCV(estimator=HistGradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4])})

In [73]:
model = gs.best_estimator_
score = cross_validate(model,x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.8871097772863686, 0.9809627539727774)

In [74]:
gs.best_params_

{'learning_rate': 0.30000000000000004}

In [75]:
from sklearn.inspection import permutation_importance


array(['alcohol', 'sugar', 'pH'], dtype=object)

In [78]:
result =  permutation_importance(model, x_train, y_train,random_state=42)
result.importances_mean

array([0.1365353 , 0.27557471, 0.14076355])

In [79]:
result =  permutation_importance(model, x_target, y_target,random_state=42)
result.importances_mean

array([0.05735385, 0.20455385, 0.05870769])

#### XGBboost --> 그레디언트 부스팅.. 알고리짐을 분산환경에 실행 할수 있도록
#### 앙상블 알고리즘.....  Bagging / Boostring
#### Bagging 각 모델에서 나온 값을 계산한다음. 최종 결과를 내는 방식
#### 부스팅 : 이전모델의 결과를 반영해서 다음 모델에 적용한후 최종 결과를 내는 방식